In [5]:
! nvidia-smi

Thu May 19 22:00:05 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 511.79       Driver Version: 511.79       CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name            TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ... WDDM  | 00000000:01:00.0 Off |                  N/A |
| N/A   68C    P0   103W /  N/A |   5199MiB /  8192MiB |     66%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [1]:
#深度学习框架都是默认在CPU上做运算，需要去指定在GPU上做运算
import torch
from torch import nn
#cpu
torch.device('cpu')

device(type='cpu')

In [2]:
#GPU,默认是0号GPU
torch.cuda.device('cuda')

In [4]:
#查询第1号gpu
torch.cuda.device('cuda:1')

In [5]:
#查询可用GPU的数量
torch.cuda.device_count()

1

In [7]:
#这两个函数允许我们在请求的GPU不存在的情况下运行代码
def try_gpu(i=0):
    if torch.cuda.device_count()>=i+1:
        return torch.device(f'cuda:{i}')
    return torch.device('cpu')

In [8]:
#返回所有可用的GPU
def try_all_gpus():  
    """返回所有可用的GPU，如果没有GPU，则返回[cpu(),]。"""
    devices = [
        torch.device(f'cuda:{i}') for i in range(torch.cuda.device_count())]
    return devices if devices else [torch.device('cpu')]

try_gpu(), try_gpu(10), try_all_gpus()

(device(type='cuda', index=0),
 device(type='cpu'),
 [device(type='cuda', index=0)])

In [9]:
#查询张量所在的设备
#默认是在cpu的内存上
x=torch.tensor([0,1,2,3])
x.device

device(type='cpu')

In [11]:
#创建的时候可以放在GPU上
x=torch.ones(2,3,device=try_gpu())
x.device

device(type='cuda', index=0)

In [12]:
#要计算x+y,我们需要决定在哪里执行这个操作
#默认是cpu上，如果我们要计算，我们需要确定X和Y在同一个设备上，同一个CPU或者同一个GPU
#这样就把x放到了1号GPU
#如果不这样想的话，可能创建网络的时候所有层没有在同一个GPU之上，那就需要来回挪数据，那网络的性能就会受到影响
z=x.cuda(1)

RuntimeError: CUDA error: invalid device ordinal

In [15]:
#神经网络与GPU
#通常在cpu上把权重初始化好
net=nn.Sequential(nn.Linear(3,1))
#手动copy一份放到GPU
net=net.to(device=try_gpu())

#0号GPU做运算
net(x)

tensor([[-0.0249],
        [-0.0249]], device='cuda:0', grad_fn=<AddmmBackward>)

In [16]:
#确认模型参数存储在同一个GPU
net[0].weight.data.device

device(type='cuda', index=0)

# 答疑

+ 需要手动copy把初始权重，输入，模型都放到GPU。这样就可以做forward,然后backward也是在GPU上
+ 一般data在network之前to gpu,因为节省GPU去训练网络，第二个是gpu不一定能很好的处理那些数据变化。如果数据读的比GPU计算快的，尽量在cpu上进行数据处理。
+ GPU上推理是指的forward
+ 默认是不共享参数的，每次实例都是新的参数。